> # **TensorFlow  2.1.0** - ***11*** - *Treinamento distribuido*

In [1]:
try:
  %tensorflow_version 2.x
except:
  pass
import tensorflow as tf
import numpy as np
import time

TensorFlow 2.x selected.


In [0]:
# carregando base de dados MNIST
(x_training, y_training), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [0]:
# normalização da base de dados
x_training = x_training / 255.
x_test = x_test / 255.

# mudança nas dimensões
x_training = x_training.reshape(-1, 28 * 28)
x_test = x_test.reshape(-1, 28 * 28)

In [6]:
# definindo modelo (não distribuido)
model_normal = tf.keras.models.Sequential()
model_normal.add(tf.keras.layers.Dense(units=128, activation='relu', input_shape=(784,)))
model_normal.add(tf.keras.layers.Dropout(rate=0.2))
model_normal.add(tf.keras.layers.Dense(units=10, activation='softmax'))
# compilando modelo normal
model_normal.compile(optimizer='adam', 
                     loss='sparse_categorical_crossentropy', 
                     metrics=['sparse_categorical_accuracy'])
model_normal.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 128)               100480    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [7]:
# definindo modelo (distribuido)
distribute = tf.distribute.MirroredStrategy()
with distribute.scope():
  model_distributed = tf.keras.models.Sequential()
  model_distributed.add(tf.keras.layers.Dense(units=128, activation='relu', input_shape=(784,)))
  model_distributed.add(tf.keras.layers.Dropout(rate=0.2))
  model_distributed.add(tf.keras.layers.Dense(units=10, activation='softmax'))
  # compilando modelo distribuido
  model_distributed.compile(optimizer='adam', 
                            loss='sparse_categorical_crossentropy', 
                            metrics=['sparse_categorical_accuracy'])

model_distributed.summary()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 128)               100480    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
______________________________________________________________

In [11]:
# comparando a velocidade entre o treinamento normal e distribuido
starting_time = time.time()
model_distributed.fit(x_training, y_training, epochs= 20, batch_size= 128)
print(f'\nDistributed training: {time.time() - starting_time}')

Train on 60000 samples
Epoch 1/20
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
60000/60000 [==============================] - 6s 106us/sample - loss: 1.8351 - sparse_categorical_accuracy: 0.5331
Epoch 2/20
60000/60000 [==============================] - 2s 33us/sample - loss: 0.9763 - sparse_categorical_accuracy: 0.7578
Epoch 3/20
60000/60000 [==============================] - 2s 35us/sample - loss: 0.6803 - sparse_categorical_accuracy: 0.8192
Epoch 4/20
60000/60000 [==============

In [12]:
starting_time = time.time()
model_normal.fit(x_training, y_training, epochs= 20, batch_size= 128)
print(f'\nDistributed training: {time.time() - starting_time}')

Train on 60000 samples
Epoch 1/20
60000/60000 [==============================] - 2s 37us/sample - loss: 1.8400 - sparse_categorical_accuracy: 0.5562
Epoch 2/20
60000/60000 [==============================] - 2s 32us/sample - loss: 0.9616 - sparse_categorical_accuracy: 0.7689
Epoch 3/20
60000/60000 [==============================] - 2s 33us/sample - loss: 0.6596 - sparse_categorical_accuracy: 0.8280
Epoch 4/20
60000/60000 [==============================] - 2s 33us/sample - loss: 0.5320 - sparse_categorical_accuracy: 0.8574
Epoch 5/20
60000/60000 [==============================] - 2s 32us/sample - loss: 0.4630 - sparse_categorical_accuracy: 0.8722
Epoch 6/20
60000/60000 [==============================] - 2s 31us/sample - loss: 0.4192 - sparse_categorical_accuracy: 0.8824
Epoch 7/20
60000/60000 [==============================] - 2s 32us/sample - loss: 0.3918 - sparse_categorical_accuracy: 0.8883
Epoch 8/20
60000/60000 [==============================] - 2s 32us/sample - loss: 0.3695 - spars